<a href="https://colab.research.google.com/github/nisarahamedk/kaggle-riid/blob/master/notebooks/RIID_TF_Transformers_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### RIID Transformer on TPU

In [56]:
import pickle

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.autograph.impl import api as autograph
from tensorflow.python.autograph.core import ag_ctx
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold


np.random.seed(42)
tf.random.set_seed(42)


### *Device* Settings - This needs to be at the TOP¶

In [57]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # no parameter needed for TPU_NAME env variable is set. This is the case for Kaggle
    print("Running on TPU: ", tpu.master())
except ValueError:
    tpu = None

Running on TPU:  grpc://10.103.175.154:8470


In [58]:
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # default strategy with the available hw
    strategy = tf.distribute.get_strategy()
    
REPLICAS = strategy.num_replicas_in_sync
print("REPLICAS: ", REPLICAS)

INFO:tensorflow:Initializing the TPU system: grpc://10.103.175.154:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.103.175.154:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


### Datasets

In [59]:
DATA_PATH = 'gs://kds-f48a9c4d95386273c0ef508e337abd3f874b82a454a6c3d0e035839c'

In [60]:
n_train_files = len(tf.io.gfile.glob(DATA_PATH + "/tfrec*"))
n_train_files

32

In [61]:
FOLDS = 10

kfold = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
folds_list = list(kfold.split(np.arange(n_train_files)))
folds_list[:2]

[(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 18,
         19, 20, 21, 22, 23, 25, 26, 27, 28, 30, 31]),
  array([15, 17, 24, 29])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 31]),
  array([ 8,  9, 25, 30]))]

In [62]:
FOLD = 1

train_folds, valid_folds = folds_list[FOLD]
len(train_folds), len(valid_folds)

(28, 4)

In [63]:
train_files = tf.io.gfile.glob([DATA_PATH + "/tfrec_%d.tfrec" % idx for idx in train_folds])
valid_files = tf.io.gfile.glob([DATA_PATH + "/tfrec_%d.tfrec" % idx for idx in valid_folds])

In [64]:
len(train_files), len(valid_files)

(28, 4)

#### Load TFRecord Datasets

In [65]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [66]:
feature_desc = {
    "content_id": tf.io.FixedLenFeature([], tf.string),
    "task_container_id": tf.io.FixedLenFeature([], tf.string),
    "prior_question_elapsed_time": tf.io.FixedLenFeature([], tf.string),
    "part": tf.io.FixedLenFeature([], tf.string),
    "prev_answered_correctly": tf.io.FixedLenFeature([], tf.string),
    "answered_correctly": tf.io.FixedLenFeature([], tf.string),
}

def parse_example(example):
  example = tf.io.parse_single_example(example, feature_desc)

  content_id = tf.io.parse_tensor(example["content_id"], tf.int16)
  task_container_id = tf.io.parse_tensor(example["task_container_id"], tf.int16)
  prior_question_elapsed_time = tf.io.parse_tensor(example["prior_question_elapsed_time"], tf.float32)
  part = tf.io.parse_tensor(example["part"], tf.int16)
  prev_answered_correctly = tf.io.parse_tensor(example["prev_answered_correctly"], tf.int8)
  answered_correctly = tf.io.parse_tensor(example["answered_correctly"], tf.int8)
  
  return tf.stack([
      tf.cast(content_id, tf.float32),
      tf.cast(task_container_id, tf.float32),
      tf.cast(prior_question_elapsed_time, tf.float32),
      tf.cast(part, tf.float32),
      tf.cast(prev_answered_correctly, tf.float32),
      tf.cast(answered_correctly, tf.float32)
  ]) # [features, seq_len] # TODO make it [seq_len, features] so that we dont have to reshape in transformer

In [67]:
def load_dataset_from_tfrecord(filenames, ds_type="train", cache_to=None):
        # Since we are reading dataset from multiple files. and we dont care about the order.
        # set deterministic reading to False.
        ignore_order = tf.data.Options()
        if ds_type == "train":
            ignore_order.experimental_deterministic = False
            
        dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE)
        if not cache_to:
            dataset = dataset.cache() # cache to RAM
        else:
            dataset = dataset.cache(cache_to) # cache to file given by self.cache_to 
        if ds_type == "train":
            # dataset = dataset.repeat() # repeat individual item, so that we have full batch at every step.
            pass
        dataset.with_options(ignore_order)
        dataset = dataset.map(parse_example, num_parallel_calls=AUTOTUNE)
        return dataset

In [68]:
dataset = load_dataset_from_tfrecord(train_files)

In [69]:
SEQ_LEN = 128

In [70]:
@tf.function
def pad(a, seq_len, max_seq_len):
  s = max_seq_len - seq_len
  # making [[0, 0], [s, 0]]
  r = tf.stack([s, tf.constant(0)])
  t = tf.stack([tf.constant([0, 0]), r])
  
  return tf.pad(a, t) # ,1 to debug

@tf.function
def trim(a, seq_len,  max_seq_len):
  start = tf.squeeze(tf.random.uniform((1,), maxval=(seq_len-max_seq_len), dtype=tf.int32))
  # https://www.quora.com/How-does-tf-slice-work-in-TensorFlow
  begin = tf.stack([tf.constant(0), start])
  size = tf.stack([tf.shape(a)[0], max_seq_len])
  
  return tf.slice(a, begin, size) # , start - to debug

@tf.function
def pad_or_trim(a):
  seq_len = tf.shape(a)[-1]
  max_seq_len = SEQ_LEN
  fn = tf.cond(tf.less_equal(seq_len, max_seq_len), lambda: pad(a, seq_len, max_seq_len), lambda: trim(a, seq_len, max_seq_len))
  return fn

In [71]:
dataset = dataset.map(pad_or_trim, num_parallel_calls=AUTOTUNE) # every sample is padded if len < SEQ_LEN or randomly trimmed to SEQ_LEN

In [72]:
@tf.function
def create_padding_mask(seq):
  seq = tf.cast(tf.reduce_all(tf.math.equal(seq, 0), axis=-1), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

@tf.function
def split_x_y_mask(xy):
  x = xy[:-1, :]
  x = tf.transpose(x, (1, 0)) # [seq_len, n_features]
  y = xy[-1, :]
  pad_mask = tf.cast(tf.math.reduce_any(tf.math.not_equal(x, 0), axis=-1), dtype=tf.float32)
  return x, tf.expand_dims(y, axis=-1), tf.expand_dims(pad_mask, axis=-1)

In [73]:
dataset = dataset.map(split_x_y_mask, num_parallel_calls=AUTOTUNE) # x and y

In [74]:
for x, y, pad_mask in dataset.take(1):
  print(x.shape)
  print(y.shape)
  print(pad_mask.shape)

(128, 5)
(128, 1)
(128, 1)


In [75]:
tf.squeeze(pad_mask, axis=-1)

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [76]:
tf.squeeze(y, axis=-1)

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 0., 0., 0., 1.], dtype=float32)>

In [77]:
dataset = dataset.shuffle(int(1024 * REPLICAS))

In [78]:
BATCH_SIZE = 256

In [79]:
dataset = dataset.batch(BATCH_SIZE)

In [80]:
for xb, yb, mb in dataset.take(1):
  print(xb.shape)
  print(yb.shape)
  print(mb.shape)

(256, 128, 5)
(256, 128, 1)
(256, 128, 1)


In [81]:
# prev_answered_feature
xb[0, :, -1]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 2., 1., 1.,
       2., 2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
       1., 1., 2., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [82]:
# corresponding mask
mb[0, :, 0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [83]:
dataset = dataset.prefetch(AUTOTUNE)

Valid dataset

In [84]:
valid_dataset = load_dataset_from_tfrecord(valid_files, ds_type="valid")
valid_dataset = valid_dataset.map(pad_or_trim, num_parallel_calls=AUTOTUNE) # every sample is padded if len < SEQ_LEN or randomly trimmed to SEQ_LEN
valid_dataset = valid_dataset.map(split_x_y_mask) # x, y and mask
valid_dataset = valid_dataset.batch(BATCH_SIZE * 2, drop_remainder=True)
valid_dataset = valid_dataset.prefetch(AUTOTUNE)

In [85]:
for vx, vy, vmask in valid_dataset.take(1):
  print(vx.shape)
  print(vy.shape)
  print(vmask.shape)

(512, 128, 5)
(512, 128, 1)
(512, 128, 1)


In [86]:
# prev_answered_feature
vx[0, :, -1]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 2., 2., 2.,
       2., 1., 2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 1., 2., 2., 1.,
       2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=float32)>

In [87]:
# corresponding mask
vmask[0, :, 0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

### Model

##### Positional Encoding

In [88]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [89]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

##### Look ahead mask

In [90]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

##### Scaled Dot Product Attention

In [91]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

##### Multi Head Attention

In [92]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

##### Pointwise FeedForward Network

In [93]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

##### EncoderLayer

In [94]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

##### Encoder

In [95]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding, embed_size_dict, rate):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.content_id_emb = tf.keras.layers.Embedding(embed_size_dict["content_id"] + 1, d_model)
    self.task_container_id_emb = tf.keras.layers.Embedding(embed_size_dict["task_container_id"] + 1, d_model)
    self.part_emb = tf.keras.layers.Embedding(embed_size_dict["part"] + 2, d_model)
    self.prior_question_elapsed_time_emb = tf.keras.layers.Dense(d_model, use_bias=True)
    self.prev_answered_emb = tf.keras.layers.Embedding(4, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)


    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embeddings and position encoding.
    c_emb = self.content_id_emb(x[..., 0])  # (batch_size, input_seq_len, d_model)
    t_emb = self.task_container_id_emb(x[..., 1])
    prior_time_emb = self.prior_question_elapsed_time_emb(tf.expand_dims(x[..., 2], axis=-1))
    pt_emb = self.part_emb(x[..., 3])
    pv_emb = self.prev_answered_emb(x[..., 4])
    x = c_emb + t_emb + prior_time_emb + pt_emb + pv_emb
    
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask) # (batch_size, input_seq_len, d_model)

    return x

In [96]:
class TransformerSeq2SeqClassifier(keras.models.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding, embed_size_dict, rate=0.1):
    super(TransformerSeq2SeqClassifier, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, maximum_position_encoding, embed_size_dict, rate)
    self.out = tf.keras.layers.Dense(1, activation="sigmoid")

  def call(self, x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = create_look_ahead_mask(seq_len)
    encoded = self.encoder(x, mask=look_ahead_mask)

    out = self.out(encoded)
    return out # [batch_size, input_seq_len, 1]


###### Embedding Sizes

In [97]:
embed_sizes = pickle.loads(tf.io.read_file(DATA_PATH + "/emb_sz.pkl").numpy())

In [98]:
embed_sizes

{'content_id': 32736, 'part': 7, 'task_container_id': 9999}

In [99]:
with strategy.scope():
  model = TransformerSeq2SeqClassifier(
      num_layers=1,
      d_model=512,
      num_heads=8,
      dff=1024,
      maximum_position_encoding=128,
      embed_size_dict=embed_sizes
  )

In [100]:
for xb, yb, mb in dataset.take(1):
  print(xb.shape, yb.shape, mb.shape)

(256, 128, 5) (256, 128, 1) (256, 128, 1)


In [101]:
# with strategy.scope():
#   y_pred = model(xb)

#### Training

In [102]:
class CustomAUC(keras.metrics.Metric):

  def __init__(self, **kwargs):
    super().__init__(**kwargs)

    self.auc = keras.metrics.AUC()

  def update_state(self, y_true, y_pred, sample_weight):

    self.auc.update_state(y_true, y_pred, sample_weight)

  def result(self):
    return self.auc.result()

  def reset_states(self):
    return self.auc.reset_states()

  def get_config(self):
    return self.auc.get_config()

  @property
  def thresholds(self):
    return self.auc.thresholds

In [103]:
# Just subclassing Loss, doesnt give us the power how to weight the sample losses.
# Hence re-implementing the __call__ method
class MaskedBCELoss(keras.losses.Loss):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

    self.bce = keras.losses.BinaryCrossentropy(reduction=keras.losses.Reduction.NONE)

  def call(self, y_true, y_pred, sample_weight):

    normal_bce_loss = self.bce(y_true, y_pred, sample_weight) # gives 0 where masked.

    # count # of non masked entries in the batch
    unmasked_count = tf.math.reduce_sum(tf.cast(tf.math.not_equal(normal_bce_loss, 0), tf.float32))

    # sum the unmasked entries.
    unmasked_sum = tf.math.reduce_sum(normal_bce_loss)
  
    average_loss = tf.math.divide(unmasked_sum, unmasked_count)

    return average_loss

  def __call__(self, y_true, y_pred, sample_weight):

    graph_ctx = tf_utils.graph_context_for_symbolic_tensors(
        y_true, y_pred, sample_weight)
    with K.name_scope(self._name_scope), graph_ctx:
      ag_call = autograph.tf_convert(self.call, ag_ctx.control_status_ctx())
      losses = ag_call(y_true, y_pred, sample_weight)
      return losses


In [104]:
# loss = MaskedBCELoss()
# loss(yb, y_pred, mb)

In [105]:
with strategy.scope():
  optimizer = keras.optimizers.Adam(learning_rate=3e-5)
  # loss = keras.losses.BinaryCrossentropy()
  loss = MaskedBCELoss()
  reduce_lr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=4, min_lr=1e-7, verbose=1)

In [106]:
with strategy.scope():
  model.compile(loss=loss, optimizer=optimizer, metrics=[keras.metrics.AUC()]) #, weighted_metrics=[CustomAUC()]) # "weighted_metrics" not supported on TPU with tf.data

In [107]:
# ## OVERFIT SINGLE BATCH
# with strategy.scope():
#   x, y, mask = next(iter(dataset.take(1))) # cannot just use take(), cause that return different batch everytime
#   print(x.shape, y.shape, mask.shape)
#   model.fit(x, y, epochs=100, sample_weight=mask, callbacks=[reduce_lr_cb])

In [108]:
with strategy.scope():
  history = model.fit(dataset, validation_data=valid_dataset, epochs=10, callbacks=[reduce_lr_cb])

Epoch 1/10
1346/1346 [==============================] - 79s 59ms/step - loss: 0.6047 - auc_2: 0.8237 - val_loss: 0.5797 - val_auc_2: 0.8156
Epoch 2/10
1346/1346 [==============================] - 66s 49ms/step - loss: 0.5778 - auc_2: 0.8150 - val_loss: 0.5710 - val_auc_2: 0.8434
Epoch 3/10
1346/1346 [==============================] - 66s 49ms/step - loss: 0.5734 - auc_2: 0.7941 - val_loss: 0.5693 - val_auc_2: 0.7564
Epoch 4/10
1346/1346 [==============================] - 67s 50ms/step - loss: 0.5714 - auc_2: 0.7945 - val_loss: 0.5718 - val_auc_2: 0.8270
Epoch 5/10
1346/1346 [==============================] - 67s 49ms/step - loss: 0.5710 - auc_2: 0.7432 - val_loss: 0.5678 - val_auc_2: 0.7925
Epoch 6/10
1346/1346 [==============================] - 66s 49ms/step - loss: 0.5693 - auc_2: 0.7652 - val_loss: 0.5754 - val_auc_2: 0.7707
Epoch 7/10
1346/1346 [==============================] - 66s 49ms/step - loss: 0.5699 - auc_2: 0.8253 - val_loss: 0.5697 - val_auc_2: 0.7431
Epoch 8/10
1346/1346

### Validation

In [109]:
with strategy.scope():
  metric = keras.metrics.AUC()

  for valid_xb, valid_yb, valid_mb in valid_dataset:
    valid_y_pred = model.predict(valid_xb)
    metric.update_state(valid_yb, valid_y_pred, valid_mb)

In [110]:
metric.result().numpy()

0.75953543

#### Save the model

In [132]:
!mkdir model

In [133]:
model.save_weights("model/best-model.h5")

Upload to kaggle

In [128]:
%%capture
!pip install kaggle

In [129]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [130]:
# Copy Kaggle API key
!mkdir -p ~/.kaggle && cp /content/drive/My\ Drive/Projects/Kaggle/api_key/kaggle.json ~/.kaggle/

In [134]:
!kaggle datasets init -p model/

Data package template written to: model/dataset-metadata.json


In [135]:
# id and title only alphanumeric and "-"
meta = """
{
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ], 
  "id": "nisarahamedk/riid-model-0",
  "title": "riid-model-0"
}
"""
with open("model/dataset-metadata.json", "w") as f:
  f.write(meta)

In [137]:
# create
!kaggle datasets create -p model/ --dir-mode tar -u

Starting upload for file best-model.h5
100% 91.6M/91.6M [00:02<00:00, 42.3MB/s]
Upload successful: best-model.h5 (92MB)
Your public Dataset is being created. Please check progress at https://www.kaggle.com/nisarahamedk/riid-model-0


#### Inference

In [116]:
new_model = TransformerSeq2SeqClassifier(
      num_layers=1,
      d_model=512,
      num_heads=8,
      dff=1024,
      maximum_position_encoding=128,
      embed_size_dict=embed_sizes
  )
new_model.build(input_shape=(128, 128, 5)) # input_shape - [batch_size, seq_len, features]
new_model.summary()

Model: "transformer_seq2seq_classifier_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_5 (Encoder)          multiple                  23991808  
_________________________________________________________________
dense_47 (Dense)             multiple                  513       
Total params: 23,992,321
Trainable params: 23,992,321
Non-trainable params: 0
_________________________________________________________________


In [117]:
new_model.load_weights("best-model.h5")

In [138]:
metric = keras.metrics.AUC()

for valid_xb, valid_yb, valid_mb in valid_dataset:
  valid_y_pred = new_model.predict(valid_xb)
  metric.update_state(valid_yb, valid_y_pred, valid_mb)

metric.result().numpy()

0.75953084